# Searching and downloading imagery from Google Earth Engine API

This notebook introduces you to the search and transfering data capabilities of GEE Python API

* The notebook uses **geetools** (developed by Rodrigo E. Principe) to display search queries on a map GUI
* It enables data **transfer** to a Google Cloud Bucket connected to the instance running the notebook



In [2]:
import ee
from ee import batch
from geetools import ui

Map = ui.Map()
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [14]:
ee.Initialize()
collection = ee.ImageCollection("COPERNICUS/S2")
## Define time range
collection_time = collection.filterDate("201-01-01", "2019-01-01")  # YYYY-MM-DD
## Select location based on location of tile
# or via geographical location:
clip = ee.Geometry.Rectangle([37.0515, -2.8504, 37.6421, -3.3194])
geometry = clip["coordinates"][0]
pathrow = collection_time.filterBounds(clip)
## Select imagery with less then 5% of image covered by clouds
clouds = pathrow.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
## Select bands
# bands = clouds.select(['B12','B11','B10','B9','B8','B8A','B7','B6','B5','B4','B3','B2','B1'])
bands = clouds.select(
    ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
)
Map.addLayer(bands, {"min": 0.0, "max": 3000.0, "gamma": [0.95, 1.1, 1]})
Map.centerObject(geometry, 9)

In [15]:
print("Export Image being submitted, please wait ...")

ee.batch.Export.image.toCloudStorage(
    image=bands.mean(),  ## mean is used here since we are trying to import a collection.
    description="Kilimanjaro_All_Bands",
    bucket="agrosight1",
    scale=30,
    maxPixels=1e9,
    region=geometry,
).start()

print("Process sent to the Cloud")

Export Image being submitted, please wait ...
Process sent to the Cloud
